<a href="https://colab.research.google.com/github/devasworski/Deeper-Networks-for-Image-Classification/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogleNet for Image Classification

Using GoogleNet on the MNIST and CIFAR datasets for image classification.

For the model definition, the code from [Khuyen Le](https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c) was used as a basis and then adjusted using the Guide from [Jai Faizy ](https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/) and the architecture of the Model was then updated to [V2](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202) 

## Imports

In [22]:
from googLeNet import googlenet
from datasets import getMnist
from datasets import getCifar

## Load Mnist

In [ ]:
def loadMnist():
    return getMnist()

## Load Cifar10

In [ ]:
def loadCifar():
    return getCifar()

## Load Model

In [20]:
model = googlenet(10)
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_137 (Conv2D)            (None, 112, 112, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_26 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_137[0][0]']             
 )                                                                                        